In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

In [3]:
all_x, all_y, points_number = get_train_data()

In [4]:
def get_resized_train_data(train_x, train_y, shape=(64, 64, 3)):
    resized_x = np.array([resize(image, shape, mode='reflect') for image in train_x])
    resized_y = np.array([[
            int(coordinate * shape[i % 2] / np.shape(image)[i % 2])
            for i, coordinate in enumerate(answer)
        ] for image, answer in zip(train_x, train_y)
    ])
    return resized_x, resized_y

In [5]:
resized_x, resized_y = get_resized_train_data(all_x, all_y)
input_shape=np.shape(resized_x[0])

In [6]:
train_x, test_x, train_y, test_y = train_test_split(
    resized_x, resized_y, test_size=0.3, random_state=42
)

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

In [8]:
datagen.fit(train_x)

In [9]:
def add_Conv2D_relu_BatchNorm(model, n_filter, filters_size, input_shape=None):
    if input_shape is None:
        model.add(Conv2D(n_filter, filters_size, padding='same'))
    else:
        model.add(Conv2D(n_filter, filters_size, padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

model = Sequential()

add_Conv2D_relu_BatchNorm(model, 64, (3, 3), input_shape)
add_Conv2D_relu_BatchNorm(model, 64, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

add_Conv2D_relu_BatchNorm(model, 128, (3, 3))
add_Conv2D_relu_BatchNorm(model, 128, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

add_Conv2D_relu_BatchNorm(model, 256, (3, 3))
add_Conv2D_relu_BatchNorm(model, 256, (3, 3))
add_Conv2D_relu_BatchNorm(model, 256, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

add_Conv2D_relu_BatchNorm(model, 512, (3, 3))
add_Conv2D_relu_BatchNorm(model, 512, (3, 3))
add_Conv2D_relu_BatchNorm(model, 512, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))

model.add(Dense(points_number * 2, activation='relu'))

In [10]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['mse'])

In [11]:
model.fit_generator(datagen.flow(train_x, train_y, batch_size=32),
                    steps_per_epoch=int(len(train_y) / 32), epochs=16)

Epoch 1/16
131/131 [==============================] - 44s - loss: 114.5259 - mean_squared_error: 114.5259    
Epoch 2/16
131/131 [==============================] - 39s - loss: 62.6738 - mean_squared_error: 62.6738    
Epoch 3/16
131/131 [==============================] - 38s - loss: 58.2655 - mean_squared_error: 58.2655    
Epoch 4/16
131/131 [==============================] - 38s - loss: 58.2270 - mean_squared_error: 58.2270    
Epoch 5/16
131/131 [==============================] - 38s - loss: 56.5218 - mean_squared_error: 56.5218    
Epoch 6/16
131/131 [==============================] - 38s - loss: 56.8916 - mean_squared_error: 56.8916    
Epoch 7/16
131/131 [==============================] - 38s - loss: 55.1748 - mean_squared_error: 55.1748    
Epoch 8/16
131/131 [==============================] - 38s - loss: 59.0262 - mean_squared_error: 59.0262    
Epoch 9/16
131/131 [==============================] - 38s - loss: 55.4514 - mean_squared_error: 55.4514    
Epoch 10/16
131/131 [=====